In [44]:
import sys
import math
import networkx as nx
from collections import OrderedDict


reads="/panfs/qcb-panasas/rdagnew/HG00514/pbmm_combined/all_3/chr6:160609100-160648600/ccs/reads.fasta"
cov=26

k=32



PSV = OrderedDict()
MG = nx.MultiDiGraph()
#MG.add_node("s")

def storePSVfilt(PSV,PSVlist,MG,k):
    l=0
    _refnode = ""
    _node = ""
    info=""
    pre=""
    for n in PSVlist:
        if n[0]=="#":
            continue  
       # print(n)
        n = n.rstrip()
        n = n.split()
      #  print(n)
        info = n[9]
        kmer = info.split(";")[4]
        pre = kmer.split("=")[1].split(",")
        _refnode = pre[0]
        _node = pre[1]
        PSV[_node] = []
        kmer = ""
#        assert k == len(line)
        for i in range(k):
            kmer = _node[i:i+k]
            PSV[_node].append(kmer)
            MG.add_node(kmer , pos=n[1], ref=_refnode[k-1], alt=_node[k-1] ,of=i,t=0)
            l=l+1
    print("stored filt PSV list:",str(l))

In [45]:
#set but need to keep track of multiplicity
PSVlist=[]
with open(psv) as p:
    PSVlist=p.readlines()
#print(PSVlist)
print(MG.number_of_nodes())
storePSVfilt(PSV, PSVlist, MG,k)
print(MG.number_of_nodes())
#print(PSV)

read_name = ""
offset = 0
starter=0
ender=0
un_read=0
read_stream = open(reads)

read_count = 0
node_multi = {}
for key in PSV.keys():
    for kmer in PSV[key]:
        node_multi[kmer]=0

0
stored filt PSV list: 2240
1325


In [46]:
for key in PSV.keys():
    for kmer in PSV[key]:
        node_multi[kmer]=0

for line in read_stream:
    if line[0] == ">":
        line = line.lstrip()
        read_name = "/".join(line.split("/")[:2])
        continue
    read = line.rstrip()
    read_len = len(read)
    first_match = 1
    read_count = read_count+1

    sentinel = "_".join(["t", read_name ])
    _starter = "_".join(["s", read_name ])
    
    
    for i in range((read_len - k)+ 1):
        kmer = read[i:i+k]


        for key in PSV.keys():
            current_off=-1
            if kmer in PSV[key]:
                offset = PSV[key].index(kmer)
            #    print(kmer)

                if MG.has_node(kmer):
                    node_multi[kmer] = node_multi[kmer] + 1
                    MG.add_node(kmer, mult=node_multi[kmer])
                else:
                    node_multi[kmer] = 1
                    MG.add_node(kmer, mult=1)


                if first_match and i == read_len - k:
                    MG.add_node(_starter,t=1)
                 #   MG.add_edge("s", _starter)
                    MG.add_node(sentinel,t=2)
                    
                    MG.add_edge(_starter, kmer, rn=read_name, of=offset, ii=i, rl= read_len)
                    MG.add_edge(kmer, sentinel, rn=read_name, ii=i, rl= read_len)
                    continue

                if first_match:
                    MG.add_node(_starter,t=1)
                   # MG.add_edge("s", _starter)
                    MG.add_edge(_starter, kmer, rn=read_name, of=offset, ii=i,rl= read_len)
                    current_match = kmer
                    current_off = offset
                    first_match = 0
                    starter = starter +1 
                    continue

                if  i == read_len - k:
                    MG.add_node(sentinel,t=2)
                    MG.add_edge(kmer, sentinel, rn=read_name, ii=i,rl= read_len)
                    ender = ender +1 
                    continue
                                                
                MG.add_edge(current_match, kmer, rn=read_name, ii=i, iof = current_off,rl= read_len, of=offset)
                current_match = kmer
                current_off = offset
                
            elif not first_match and i == read_len - k:
                MG.add_node(sentinel, t=2)
                MG.add_edge(current_match, sentinel, rn=read_name, ii=i , iof = current_off,rl= read_len)
                ender = ender +1 

    if first_match:
        un_read = un_read+1            
    if read_count%10 == 0:
        #print(read_count)
        print(MG.number_of_edges())
    if read_count%100 == 0:
        print(read_count)




1396
4330
5820
6724
7542
8417
9239
10136
10917
11833
100
12704
13650
14506
15418
16194
16787
17569
18319
19031
19836
200
20621
21767
22965
24328
25608
26859
28551
29831
30539
31274
300
32640
35286
37012
38732


In [47]:
print(MG.number_of_edges())
print(MG.number_of_nodes())

38732
2001


In [48]:
print("reads: ",str(read_count),"discard reads: ",str(un_read),"\nretain nodes: ",str(MG.number_of_nodes()))#,"discard nodes: ",str(un_node))
print("source: ",str(starter),"\nsink: ",str(ender))

reads:  345 discard reads:  7 
retain nodes:  2001
source:  338 
sink:  14196


In [49]:
ini=1

master=""
prev=""

for node in MG.nodes():#.nodes():
    if ini:
        master=node
        prev=node
 #       print(prev)
#        deg= MG.out_degree(node)
        ini=0
    elif not ini:
#        print("enter1",prev[1:],node[:-2])      
        if str(prev[1:]) == str(node[:-1]):
     #       print("enter")
            MG = nx.contracted_nodes(MG,master,node)
            prev=node
        else:
            ini=1

In [50]:
#MG.nodes()
print(MG.number_of_edges())
print(MG.number_of_nodes())
print(len(PSVlist))

38841
759
70


In [51]:
un_node=0
rem=[]
for node in MG.nodes():
    if MG.degree(node) == 0:
        rem.append(node)
        un_node = un_node+1
MG.remove_nodes_from(rem)
print(len(rem))

6


In [52]:
#MG.nodes()
print(MG.number_of_edges())
print(MG.number_of_nodes())
print(len(PSVlist))

38841
753
70


In [53]:
out="/home/cmb-16/mjc/rdagnew/summerproj/kmer_filter/psv_graph/psv.COMB.clr_filt.gml"

nx.write_gml(MG,out)